### Measuring the correlation between categorical variables Provider Tenure and  Medication Given

This notebook analyzes and describes relationship that exists( or not) between provider tenure and EMS treatment(Medication Given) to the patient. The script reads the intermediate dataset -- 'MedicationsPatients' preapared by our team based on datasets provided by our project partner.

Initially, in this classification situation, catogorical target variable 'Medication_Given' and categorical predicator 'Provider's Tenure'(in months & years), are analyzed and the strength of relationship between them is measured using <B>Chi-square test<B>.
    
Then the more details analysis of the relationship between provider's tenure and Medication Given is done including various graphical/visual representations

<b>Chi-square Test of Independence:<b>
<p>The chi-square test of independence is used to determine whether there is an association between two or more categorical variables. In our case, we would like to test whether the Tenure of the provider has any association with Medication they administered.

##### This note book uses Intermediate Datasets -- MedicationsPatients for analytics

In [ ]:
# import the libraries needed
import pandas as pd
import numpy as np
import os, time

# Used for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print(os.getcwd())
readStart=time.time()

## Reading dataset
columnsToUse=['PatientId','PatientGender','PatientGenderCode',
            'FRDPersonnelID','FRDPersonnelGender','ProviderGenderCode',
            'Medication_Given_RXCUI_Code','Medication_Given_Description','TenureMonths']

dfMedPatient = pd.read_csv ('../data/02_intermediate/MedicationsPatients-20210225-ems-raw-v04.csv',usecols=columnsToUse)[columnsToUse]

# Stop the clock and calculate read time
readStop=time.time()
readTime=readStop-readStart
readMin=np.floor(readTime/60)
readSec=np.floor(readTime-(readMin*60))
print("The file was read in {0:.0f} minutes and {1:.0f} seconds.".format(readMin,readSec))

print(dfMedPatient.shape)

In [ ]:
# list the columns in the dataframe
dfMedPatient.columns

In [ ]:
#Confirming Given Medication Names
##print('Medication Given( Unique ) =\n '+str(dfMedPatient.Medication_Given_Description.unique()))

In [ ]:
#Checking Medication Given Description count
print('Medication Given Description Count = '+str(dfMedPatient.Medication_Given_Description.nunique()))
##dfMedPatient.Medication_Given_Description.nunique()

In [ ]:
dfMedPatient["Medication_Given_Description"].value_counts()

creating output to generate an MedicationDescriptionOrder to be used in plots later.

In [ ]:
#Put the value_counts into a dataframe for later use
value_countsMedGiven=dfMedPatient['Medication_Given_Description'].value_counts()
df_MedicationGivenDescription=pd.DataFrame(value_countsMedGiven)
df_MedicationGivenDescription.reset_index(inplace=True)
df_MedicationGivenDescription.columns=["Medication_Given_Description","Count"]
df_MedicationGivenDescription.set_index(keys=["Medication_Given_Description"])
df_MedicationGivenDescription

In [ ]:
#Create the MedicationGivenOrder array from the newly created dataframe
MedicationGivenOrder=np.array(df_MedicationGivenDescription["Medication_Given_Description"])
MedicationGivenOrder

In [ ]:
#Confirming unique Medication_Given_RXCUI_Code count
print('Medication_Given_RXCUI_Code Count = '+str(dfMedPatient.Medication_Given_RXCUI_Code.nunique()))

Removing 'Oxygen' and 'Normal saline' from 'MedicationGivenOrder' array for later sorting /indexing purpose


In [ ]:
#Removing MedicationGivenOrder 'Oxygen' and 'Normal saline' from 'MedicationGivenOrder' numpy array using 'numpy.delete(array, index)'

## index : 0='Oxygen' ; 1 ='Normal saline'
index = [0, 1]
MedicationGivenOrderWithoutOxySaline= np.delete(MedicationGivenOrder,index)
MedicationGivenOrderWithoutOxySaline

Confirmed that the unique Medication_Given_Description coun matches the Medication_Given_RXCUI_Code count of <b>32.</b>

In [ ]:
#Confirming Tenure Months(unique) in sorted order
print('Tenure Months list =\n '+str(dfMedPatient.TenureMonths.sort_values().unique()))
##dfMedPatient.TenureMonths.sort_values().unique()

In [ ]:
#Show number of unique values per column
dfMedPatient.nunique(dropna=False)

In [ ]:
#Show number of nulls per column
dfMedPatient.isnull().sum()

Dispalying Time Traveler - Providers ( TenureMonths in future (<0 months) )

In [ ]:
# showing dfMedPatient for those records that have a negative tenure ( time travellers)
dfMedPatient[(dfMedPatient['TenureMonths'] < 0)]

### Reduce Data

Going to drop the "time traveler" (negative tenure) rows

In [ ]:
#Remove -ve Tenure for PatientProvider ( Removing time traveller records)
dfMedPatientReduced = dfMedPatient[(dfMedPatient['TenureMonths'] >= 0)]

#Calculate percentage of dataset remaining
ratio=len(dfMedPatientReduced)/len(dfMedPatient)*100
print("%.4f%% remaining!" % ratio)

<B> Now adding a new column 'TenureYears' for further analytics </b>.Here we simply divide the TenureMonths by 12 and drop the remainder.
Calculation of Tenure in Months results in value that represents number of completed years of service. (e.g. 35 months -> 2 years, 36 months -> 3 years)

In [ ]:
dfMedPatientReduced.loc[:,"TenureYears"]=np.floor(dfMedPatientReduced["TenureMonths"]/12)

In [ ]:
dfMedPatientReduced.head()

In [ ]:
# Cross tabulation between  TenureMonths and Medications from 'dfMedPatientReduced' datarframe  -- Keeping this code line from -cross-validation/checking----
CrosstabResult=pd.crosstab(index=dfMedPatientReduced['TenureMonths'],columns=dfMedPatientReduced['Medication_Given_Description'])
##print(CrosstabResult)

Chi-square Test in Python can be done using the chi2_contingency() function from the scipy.stats module.

In [ ]:
# importing the required function
from scipy.stats import chi2_contingency

In [ ]:
# Cross tabulation between TenureMonths and Medications from 'dfMedPatientReduced' datarframe 
pd.crosstab(dfMedPatientReduced.Medication_Given_Description,dfMedPatientReduced.TenureMonths)

Chi-square test finds the probability of a Null hypothesis(H0).

 - Assumption(H0): The two variables are NOT related to each other
 - Result of Chi-Sq Test: if the Probability of H0 being True, The two variables are NOT related to each other
<p>It can help us to understand whether both the categorical variables are correlated with each other or not.

In [ ]:
## Performing Chi-sq test, to test the association between two variables, using the cross tab sequence
##ChiSquareResult=chi2_contingency(pd.crosstab( dfMedPatientReduced.TenureMonths, dfMedPatientReduced.Medication_Given_Description))
ChiSquareResult=chi2_contingency(pd.crosstab( dfMedPatientReduced.TenureMonths, dfMedPatientReduced.Medication_Given_RXCUI_Code))

In [ ]:
# P-Value is the Probability of H0 being True
# If P-Value>0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSquare Test is:', ChiSquareResult[1])

As evident, the <b>p-value is less than 0.05 </b>, hence we <b>reject the Null Hypothesis(H0) that the 'TenureMonths' of the Providers are not associated with the 'Medication_Given'</b>

As the P-value came lower than 0.05 in our result, hence H0 will not be accepted, which means the <b>variables 'TenureMonths' and 'Medication_Given' are correlated  to each other</b>. This is based on if two variables are correlated, then the P-value will come very close to zero, which in our case is : <b>4.5781793145246635e-174</b>

<b>Performing Chi-square test with TenureYears values </b>

In [ ]:
## Performing Chi-sq test with TenureYears, to test the association between two variables, using the cross tab sequence
ChiSquareResultforTYears=chi2_contingency(pd.crosstab( dfMedPatientReduced.TenureYears, dfMedPatientReduced.Medication_Given_RXCUI_Code))

In [ ]:
# P-Value is the Probability of H0 being True
# If P-Value>0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSquare Test( Teure Years) is:', ChiSquareResultforTYears[1])

<b>Now, further investigating relationship between the provider's TenureMonths and Medications Given follows </b>:........

Create a pivot-like dataframe using tenure, Medication Code and  Medication Description to get a Medication Given count break down by tenure and medication.

In [ ]:
## Creating pivot dataframe from'dfMedPatientReduced' 
dfMedPatientReduced_pvt = dfMedPatientReduced.groupby(['TenureMonths','TenureYears',
                                    'Medication_Given_RXCUI_Code',
                                    'Medication_Given_Description']).size().to_frame(name='Medication_Count').reset_index()

dfMedPatientReduced_pvt.shape

<b>What are the top 10 Medication Given overall? </b>

In [ ]:
dfMedPatientReduced_pvt.groupby(['Medication_Given_RXCUI_Code',
                          'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

<b> Removing data where Medication_Given_Description = 'Oxygen' and 'Normal saline' from dfMedPatientReduced dataframe, before 2nd round of Chi-Square Test  </b>

In [ ]:
#Remove 'oxygen' for 'Medication_Given_Description'
dfMedPatientReduced_NoOxy=dfMedPatientReduced.loc[dfMedPatientReduced['Medication_Given_Description']!='Oxygen',:]

#size of remaining dataset 
len(dfMedPatientReduced_NoOxy)

In [ ]:
#Remove 'Normal saline' for 'Medication_Given_Description'
dfMedPatientReduced_NoOxySaline=dfMedPatientReduced_NoOxy.loc[dfMedPatientReduced_NoOxy['Medication_Given_Description']!='Normal saline',:]

#size of remaining dataset 
len(dfMedPatientReduced_NoOxySaline)

In [ ]:
# Now again creating Cross tabulation between TenureYears(this time) and Medications from 'dfMedPatientReduced_NoOxySaline' datarframe 
pd.crosstab(dfMedPatientReduced_NoOxySaline.Medication_Given_Description,dfMedPatientReduced_NoOxySaline.TenureYears)

In [ ]:
## Now Performing Chi-sq test, without 'oxygen' and 'Normal saline' and 'TenureYears' to test the association between two variables, using the cross tab sequence
ChiSquareResult_NoOxySaline=chi2_contingency(pd.crosstab(dfMedPatientReduced_NoOxySaline.TenureYears, dfMedPatientReduced_NoOxySaline.Medication_Given_Description))

In [ ]:
# P-Value is the Probability of H0 being True
# If P-Value>0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSquare Test is:', ChiSquareResult_NoOxySaline[1])

Again, we see the <b>p-value is less than 0.05 </b>, hence we <b>reject the Null Hypothesis(H0) that the 'TenureMonths' of the Providers are not associated with the 'Medication_Given'</b>

<b>Using dfMedPatientReduced_pvt for further analysis</b>

In [ ]:
## Changing 'Medication_Given_RXCUI_Code' from Numerical(float) to int64 for consistency
dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'] = dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'].apply(np.int64) 

In [ ]:
#Create the MedicationOrder array from the dataframe
MedicationOrder=np.array(dfMedPatientReduced["Medication_Given_Description"])
MedicationOrder

In [ ]:
## Lowest 3 tenure months Medication given, count break down by tenuremonths
dfMedPatientReduced_pvt.head(3)

In [ ]:
## Top 3 tenuremonths for Medication given, count break down by tenuremonths
dfMedPatientReduced_pvt.tail(3)

Generate a hex bin plot using tenure, medication given code, and medication count from dfMedPatient_pvt as the x, y and C values, respectively.

In [ ]:
# in order to get the medication given code to show in the y-axis, need to format the values as strings instead of numbers
hb = dfMedPatientReduced_pvt.plot.hexbin(
                       x='TenureYears', 
                       y='Medication_Given_RXCUI_Code', 
                       C='Medication_Count',
                       reduce_C_function=np.sum,
                       gridsize=110,
                       cmap="nipy_spectral",
                       xlabel="Provider Tenure(Years)",
                       ylabel="Medication Code",
                       title="Provider Tenure and Medication Given (dfMedPatientReduced_pvt)",
                       figsize=(16,14)
                       ##sharex=False
)
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:.0f}'))
plt.show

The hex bin plots of dfMedPatientReduced_pvt shows the use of 3 attributes from the dataframe. Using x, y, and C means that the reduce_C_function now comes into play and it is the reason for all of the white showing - it is where there is no sum to show.

Using dfMedPatientReduced_pvt, generate descriptive statistics about the tenure, medicine count, and medication given description attributes.

In [ ]:
dfMedPatientReduced_pvt[['TenureYears','Medication_Count']].describe()

In [ ]:
plt.figure(figsize=(18,20))
##sns.set_theme(style="whitegrid")
sns.violinplot("TenureYears","Medication_Given_Description",
               data=dfMedPatientReduced[dfMedPatientReduced.TenureYears < 40], 
               scale='width',
              order=MedicationGivenOrder, cut=0, palette="muted", inner="quartile");


In [ ]:
##plt.title('Medication Given vs Tenure Violinplot')
plt.figure(figsize=(18,20))
##sns.set_theme(style="whitegrid")
sns.violinplot(x="TenureYears",y="Medication_Given_Description",
               data=dfMedPatientReduced, hue="FRDPersonnelGender", split=True, dodge =True,
               scale='area',
              order=MedicationGivenOrder, cut=0, palette="muted", inner="quartile", height=15, aspect=.8);

<b>Now, Creating violin plot using dataframe without Generally used medicine 'Oxygen' & 'Normal Saline' with high skewness for comparision</b>

In [ ]:
## Using dataframe 'dfMedPatientReduced_NoOxySaline' removing medicine 'Oxygen' & 'Normal Saline' for violin plot 

plt.figure(figsize=(18,20))
##sns.set_theme(style="whitegrid")
sns.violinplot(x="TenureYears",y="Medication_Given_Description",
               data=dfMedPatientReduced_NoOxySaline, hue="FRDPersonnelGender", split=True, dodge =True,
               scale='area',
              order=MedicationGivenOrderWithoutOxySaline, cut=0, palette="Set2", inner="quartile", height=15, aspect=.8);

Now,looking for more details on the data frame

In [ ]:
dfMedPatientReduced_pvt[['Medication_Given_Description']].describe(include='all')

Generate histograms of the tenure and medication given using all records to see frequency from dfMedPatient_pvt

In [ ]:
## histogram of the tenure
dfMedPatientReduced.hist(column='TenureYears',bins=40)

Finding the distributions for tenure years of the providers(also gender)

In [ ]:
sns.violinplot(x="TenureYears",y="FRDPersonnelGender",
               data=dfMedPatientReduced,
               palette="muted", split=True,
               scale='count', cut=0);

In [ ]:
##Barplot of the 'Medication_Given_Description' and total given count
##df_MedicationGivenDescription.plot.barh(x='Medication_Given_Description', y='Count', color='red', align='center', alpha=0.7, figsize=(20,12))

<b>Reviewing distribution of Medication_Given_Description </b>

In [ ]:
## bar plot for medication given distribution 
fig=plt.figure(figsize=(20,14))
value_counts=dfMedPatientReduced['Medication_Given_Description'].value_counts()
df_GivenMedicationByProviders=pd.DataFrame(value_counts)
df_GivenMedicationByProviders.reset_index(inplace=True)
df_GivenMedicationByProviders.columns=["Medication_Given_Description","Count"]

#Bar chart
ax1=plt.subplot()
ax1.barh(df_GivenMedicationByProviders['Medication_Given_Description'],width=df_GivenMedicationByProviders['Count'],color='purple')
ax1.set_xlabel('# of Given Medications ')
ax1.grid(axis='x', alpha=.75, which='both')

ax1.set_xticks(minor=True, ticks=[100,200,300,400,600,700,800,900,1100, 1200,1300,1400])
ax1.set_xticks(minor=False, ticks=[500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500])
                                  
fig.suptitle('Given Medication Distribution')
plt.show()

In [ ]:
## !! Check, if this is valid !!! histogram of the 'Medication_Count' (by tenure)
dfMedPatientReduced_pvt.hist(column='Medication_Count', bins=40)

Please remember there are some "time traveler" tenure values (2 records) where the porvider start date is after the dispatch date resulting in a negative value for the calculated provider tenure. Those records were removed for analysis.

<b>What are the top 10 Medication Given overall? </b>

In [ ]:
dfMedPatientReduced_pvt.groupby(['Medication_Given_RXCUI_Code',
                          'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

Based on the tenure histogram, the highest tenure count  falls between the 0 years (< 1 year) and 20 years value(mean tenure 13.37 years) . Now going to look at provider tenure values less than or equal to 20 years tenure to see what can be found around these "high" range.

In [ ]:
# how many records from dfMedPatientReduced_pvt will be used limiting the tenure to 100 months or less
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)].shape

<b>Generate the respective descriptive statistics and histograms. </b>

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)][['TenureYears','TenureMonths','Medication_Count']].describe()

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)][['Medication_Given_Description']].describe(include='all')

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)].hist(column='Medication_Given_RXCUI_Code', bins=40)

A comparison of the medication given histograms for all dfMedPatientReduced_pvt rows and the subset limited to tenure of 20 years or less shows the distribution of medication given is seemingly distributed evenly except for certain medications.

The mean of the tenure values for those providers with 20 years or less, is <b>9.46 </b> years, as seen above earlier.

<b> What are the top 10 procedures by count for medication given by providers with 20 or less years of tenure? </b>

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)].groupby(['Medication_Given_RXCUI_Code',
     'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

In [ ]:
# create a dataframe for for medication given by providers with 20 or less years of tenure for repeated use in analysis
dfMedPatientReduced_pvt_lte20Yrs= dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)]
dfMedPatientReduced_pvt_lte20Yrs.shape

<b>What are the top 10 procedures by count for procedures performed by providers with more than 20 or more years of tenure? </b>

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] > 20)].groupby(['Medication_Given_RXCUI_Code',
     'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

In [ ]:
# create a dataframe for medication given by providers with more than 20 years of tenure for repeated use in analysis
dfMedPatientReduced_pvt_gt20Yrs= dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] > 20)]
dfMedPatientReduced_pvt_gt20Yrs.shape

<b>Generate the respective descriptive statistics and histograms for medication given by providers with more than 20 years of tenure. </b>

In [ ]:
# describe for dataframe dfMedPatientReduced_pvt_gt20Yrs
dfMedPatientReduced_pvt_gt20Yrs[['TenureYears','TenureMonths','Medication_Count']].describe()

In [ ]:
# describe for 'Medication_Given_Description'
dfMedPatientReduced_pvt_gt20Yrs[['Medication_Given_Description']].describe(include='all')

The medication histogram for providers with 20 years or less and greater than 20 years continues to display a similar distribution pattern of medication given (even scales are less)

In [ ]:
# show procedure histograms from all tenures, tenure < = 20 years and  tenure > 300 years side by side

fig=plt.figure(figsize=(12,4))

# all tenure
ax1=plt.subplot(121)
ax1.hist(x=dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'],bins= 40, alpha = 0.5)
ax1.set_xlabel('Medication Given Code')
ax1.set_ylabel('Medication Count')
ax1.set_title('Medication Given by All Providers')
ax1.ticklabel_format(style='plain')
ax1.grid()
plt.xticks(rotation=45)

# lte >=20 years tenure
ax2=plt.subplot(122)
ax2.hist(x=dfMedPatientReduced_pvt_lte20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='red')
ax2.set_xlabel('Medication Given Code')
ax2.set_ylabel('Medication Count')
ax2.set_title('Medication Given by Providers with less than 20 years tenure')
ax2.ticklabel_format(style='plain')
ax2.grid()
plt.xticks(rotation=45)

# show the first 2 plots
plt.show()

fig=plt.figure(figsize=(12,4))

# gt > 30 years tenure
ax1=plt.subplot(121)
ax1.hist(x=dfMedPatientReduced_pvt_gt20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='purple')
ax1.set_xlabel('Medication Given Code')
ax1.set_ylabel('Medication Count')
ax1.set_title('Medication Given by Providers with more than 20 years tenure')
ax1.ticklabel_format(style='plain')
ax1.grid()
plt.xticks(rotation=45)

# show all the plots
plt.show()


Now showing the histograms as overlays so thie comparision can be seen  on the same scale

In [ ]:
## overlays histograms 
fig=plt.figure(figsize=(16,8))

ax1=plt.subplot(121)
ax1.hist(x=dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'],bins= 40, alpha = 0.5, label='all tenures')
ax1.hist(x=dfMedPatientReduced_pvt_lte20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='red', label='<= 20 years')
ax1.hist(x=dfMedPatientReduced_pvt_gt20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='purple', label='> 20 years')
ax1.set_xlabel('Medication Given Code')
ax1.set_ylabel('Medication Count')
ax1.set_title('Medication Given by Providers')
ax1.ticklabel_format(style='plain')
ax1.grid()
plt.xticks(rotation=45)
plt.legend(loc='upper right')
plt.show()


<b>Now looking for the following medications which has adminstered by providers with specific tenures </b>
  (Medication Name    Code     Group)
 - Glucagon (Glucagen)- 4832 - Diabetic
 - Magnesium Sulfate (50%) - 6585 - Severe Asthma/COPD
 - Tranexamic Acid (TXA) - 10691 - Trauma
 - Lidocaine (2%) (Xylocaine) - 6387 - Pain
 - Morphine (Morphine Sulfate) - 7052 - Pain
 - Norepinephrine (Levophed) - 328853 - Hypotension
 - Calcium Chloride (10%) - 1901 - Cardiac
- Tetracaine - 91189 - Eye Pain due to trauma
 - Cyanokit (Hydroxocobalamin) - 5514 - Poisoning



In [ ]:
dfMedPatientReduced_Filtered = dfMedPatientReduced[dfMedPatientReduced.Medication_Given_Description.isin(["Glucagon (Glucagen)","Magnesium Sulfate (50%)", "Tranexamic Acid (TXA)", "Lidocaine (2%) (Xylocaine)","Morphine (Morphine Sulfate)", "Norepinephrine (Levophed)","Calcium Chloride (10%)", "Tetracaine","Cyanokit (Hydroxocobalamin)"])]
dfMedPatientReduced_Filtered.head()


In [ ]:
dfMedPatientReduced_Filtered.size

### Findings
Question: Is there a relationship between category(name) of medication given and the provider tenure?
Answer: There seems to be NO significant indicators of a relationship or anomaly associated with provider tenure and the medication given.

<b>Conclusions / Observations </b>
 - The distribution of medication given is nominally similar across all tenures (by year), with some slight variation.
 - The distribution of tenures (by year) is nominally similar across all medication given, with with some slight variation.
 - These distributions are consistent with the distributions of their underlying factors.

